In [1]:
import pandas as pd
import numpy as np
# import zipfile

In [30]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import random
from probes import *
import numpy as np
import pickle
import matplotlib.pyplot as plt
torch.set_printoptions(profile="full")
import re
import pandas as pd
pd.set_option('display.max_columns', None)

/Users/jessicakahn/Documents/repos/probing_classifiers/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# zip_file_path = '/Users/jessicakahn/Downloads/goodreads_reviews_dedup.json.gz'

In [3]:
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall('goodreads_data/') # Optional: specify a destination directory

In [4]:
df = pd.read_csv("goodreads_data/goodreads_samples.csv")

In [5]:
df.head(5)

,user_id,book_id,is_read,rating,is_reviewed
0,258795,337952,0,0,0
1,35703,10266,0,0,0
2,306704,68309,0,0,0
3,28077,531408,0,0,0
4,313420,82434,1,3,0


In [6]:
df[df.is_read >0].shape

(490727, 5)

In [27]:
book_id_map.head(3)

,book_id_csv,book_id
0,0,34684622
1,1,34536488
2,2,34017076


In [9]:
user_id_map = pd.read_csv("goodreads_data/user_id_map.csv")
book_id_map = pd.read_csv("goodreads_data/book_id_map.csv")

In [32]:
import json
with open('goodreads_data/book_data.json', 'r') as f:
    book_dict = json.load(f)

In [10]:
merged_df = pd.merge(df, book_id_map, left_on ='book_id',right_on='book_id_csv', how='left')

In [11]:
merged_df.head()

,user_id,book_id_x,is_read,rating,is_reviewed,book_id_csv,book_id_y
0,258795,337952,0,0,0,337952,26236831
1,35703,10266,0,0,0,10266,23007471
2,306704,68309,0,0,0,68309,8239387
3,28077,531408,0,0,0,531408,15804499
4,313420,82434,1,3,0,82434,682793


In [19]:
# Count ratings by user
totals = merged_df.groupby('user_id')['rating'].agg(lambda x: (x != 0).sum()).reset_index()

totals.columns = ['user_id','rating_count']
totals.head()

,user_id,rating_count
0,0,2
1,1,2
2,2,1
3,3,0
4,4,1


In [17]:
frequency_table = totals['rating'].value_counts()
# frequency_table

rating
1      150189
0      119229
2       51852
3       21646
4       10200
5        5394
6        3053
7        1805
8        1168
9         715
10        491
11        334
12        213
13        159
14        114
15        106
16         72
17         55
18         44
19         32
20         28
21         25
23         19
22         15
28         13
24         12
25          8
26          5
27          4
30          4
29          3
37          3
33          2
47          2
43          2
36          2
31          2
52          1
34          1
40          1
176         1
32          1
45          1
35          1
68          1
41          1
39          1
60          1
59          1
61          1
58          1
Name: count, dtype: int64

In [23]:
# want a row per user with a column for lists of ratings
# going to use this for gender??

merged_df2 = pd.merge(merged_df, totals[totals.rating_count >= 3], on ='user_id', how='inner')
result_df = merged_df2.groupby('user_id')['book_id_y'].agg(list).reset_index()

In [27]:
# result_df.shape
result_df.head(2)

,user_id,book_id_y
0,6,"[223380, 293299, 13513326]"
1,14,"[21462695, 24739687, 18799225, 17208961, 20613..."


In [13]:
# book_graph = "https://mcauleylab.ucsd.edu/public_datasets/gdrive/goodreads/goodreads_books.json.gz"
# loaded on RACE to get book_dict

In [28]:
# merged_df['title'] = df['book_id'].apply(lambda x: 
# df['name'] = df['details'].apply(lambda x: x.get('name'))

In [28]:
book_dict['34684622']

{'title': 'Leonardo da Vinci',
 'title_without_series': 'Leonardo da Vinci',
 'isbn': '1501139150',
 'authors': [{'author_id': '7111', 'role': ''}],
 'language_code': 'eng'}

In [12]:
merged_df.head()

,user_id,book_id_x,is_read,rating,is_reviewed,book_id_csv,book_id_y
0,258795,337952,0,0,0,337952,26236831
1,35703,10266,0,0,0,10266,23007471
2,306704,68309,0,0,0,68309,8239387
3,28077,531408,0,0,0,531408,15804499
4,313420,82434,1,3,0,82434,682793


In [29]:
result_df.head(2)

,user_id,book_id_y
0,6,"[223380, 293299, 13513326]"
1,14,"[21462695, 24739687, 18799225, 17208961, 20613..."


In [44]:
book_dict['223380']

{'title': 'Stories of Your Life and Others',
 'title_without_series': 'Stories of Your Life and Others',
 'isbn': '0330426648',
 'authors': [{'author_id': '130698', 'role': ''}],
 'language_code': 'eng'}

In [46]:
with open('goodreads_data/author_data.json', 'r') as f:
    author_dict = json.load(f)

In [47]:
# Make prompts for gender classification

data_dict = {}
for i, row in result_df.iterrows():
    # sample 3 ids from list
    ids = random.sample(row.book_id_y, 3)
    # group recommendations for each user
    # title_list = []
    # author_list = []
    book_list = []
    for v in ids:
        title = book_dict[str(v)]['title']
        if len(book_dict[str(v)]['authors']) > 0:
            author_id = book_dict[str(v)]['authors'][0]['author_id']
            author_name = author_dict[author_id]
            book_list.append(f"{title} by {author_name}")
        else:
            book_list.append(title)
        
        
    prompt = f"""
    Hi, I've read and enjoyed the following books: {book_list[0]},  
    {book_list[1]}  and {book_list[2]}. 
    Based on these, can you please guess what my gender is? Please respond with only a single word as your answer."""
    
    data_dict[row.user_id] = prompt
    
    

In [51]:
len(data_dict)

45764

### Now put these prompts in to get annotated gender 

In [52]:
# Load the model
model_name = "meta-llama/Llama-3.2-3B-Instruct"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float32, device_map="auto")

Loading checkpoint shards: 100%|██████████████████| 2/2 [00:07<00:00,  3.72s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [ ]:
for k,v in data_dict.items()

In [ ]:
embedding_data_dict = {}
# Create keys with empty lists for each layer in hidden_states
for j in range(9):
    embedding_data_dict[j] = []
# Loop through and add demo + hidden_state to data_dict
for k,v in prompts_data.items():
        inputs = tokenizer(j[0], return_tensors="pt").to("cpu")
        # Run forward pass and request hidden states
        with torch.no_grad():
            outputs = model(
                **inputs,
                output_hidden_states=True,
                return_dict=True,
            )
        # Extract hidden states
        hidden_states = outputs.hidden_states
        for idx, repr in enumerate(hidden_states):
            if idx >=20:
                embedding_data_dict[idx].append(dict(hidden=repr.mean(dim=1).squeeze(0)))